In [1]:
import sys
sys.path.append("./live_tools")
import ccxt
import ta
import pandas as pd
from utilities.perp_bitget import PerpBitget
from utilities.custom_indicators import get_n_columns
from datetime import datetime
import time
import json


ModuleNotFoundError: No module named 'utilities'

In [ ]:
f = open(
    "api_connection.json")
api_connection = json.load(f)
f.close()

account_to_select = "bitget_connection"
production = True

In [ ]:
bitget = PerpBitget(
    apiKey=api_connection[account_to_select]["apiKey"],
    secret=api_connection[account_to_select]["secret"],
    password=api_connection[account_to_select]["password"],
)

In [4]:
api_connection[account_to_select]["password"],

('Moscareta28011Imagination1',)

In [6]:
usd_balance = float(bitget.get_usdt_equity())
usd_balance

157.718067802639

In [ ]:
import pandas as pd
import numpy as np
import requests
import json
import time
from datetime import datetime
import os


#Path = os.path.dirname( os.path.abspath(__file__))
def btc():
    url = "https://www.bitstamp.net/api/v2/ohlc/btcusd/"
    timeframe = 86400
    start_date = 1312174800
    end_date = int(time.time())

    # set the maximum number of data points to return per request
    limit = 1000

    data_list = []
    while start_date <= end_date:

        request_end_date = min(start_date + limit*timeframe, end_date)
        response = requests.get(url + "?step=" + str(timeframe) + "&start=" + str(start_date) + "&end=" + str(request_end_date) + "&limit=" + str(limit))
        data = json.loads(response.text)
        data_list += data['data']['ohlc']
        start_date += limit*timeframe

    df = pd.DataFrame(data_list)
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s')
    df[['open', 'high', 'low', 'close', 'volume']] = df[['open', 'high', 'low', 'close', 'volume']].astype(float)

    df.set_index('timestamp',drop=True, inplace = True)
    #df.columns = [e.capitalize() for e in df.columns]
    df.sort_index(ascending = True, inplace= True)


    df.drop_duplicates(inplace=True)
    # Add EHMA indicator
    period = 20
    sqrt_period = np.sqrt(period)

    def John_Ehlers(x, y):
        alpha = 2 / (y + 1)
        sum = np.zeros(len(x))
        sum[0] = alpha * x[0]
        for i in range(1, len(x)):
            sum[i] = alpha * x[i] + (1 - alpha) * sum[i-1]
        return sum

    close_ema1 = John_Ehlers(df['Close'], int(period / 2))
    close_ema2 = John_Ehlers(df['Close'], period)
    ehma = John_Ehlers(2 * close_ema1 - close_ema2, sqrt_period)

    df['ehma'] =ehma

    df['ehma_1'] = df['ehma'].shift(1)

    return df


In [ ]:
df = btc().copy()

In [ ]:
df.shape

In [ ]:
df

In [ ]:

pair = "BTC/USDT:USDT"
timeframe = "1h"
leverage = 1

print(f"--- {pair} {timeframe} Leverage x {leverage} ---")

type = ["long", "short"]
bol_window = 100
bol_std = 2.25
min_bol_spread = 0
long_ma_window = 500


bitget = PerpBitget(
    apiKey=secret[account_to_select]["apiKey"],
    secret=secret[account_to_select]["secret"],
    password=secret[account_to_select]["password"],
)

# Get data
df = bitget.get_more_last_historical_async(pair, timeframe, 1000)

# Populate indicator
df.drop(columns=df.columns.difference(['open','high','low','close','volume']), inplace=True)
bol_band = ta.volatility.BollingerBands(close=df["close"], window=bol_window, window_dev=bol_std)
df["lower_band"] = bol_band.bollinger_lband()
df["higher_band"] = bol_band.bollinger_hband()
df["ma_band"] = bol_band.bollinger_mavg()

df['long_ma'] = ta.trend.sma_indicator(close=df['close'], window=long_ma_window)

df = get_n_columns(df, ["ma_band", "lower_band", "higher_band", "close"], 1)


In [ ]:
df

In [ ]:
df.iloc[-1]

In [7]:
positions_data = bitget.get_open_position()
position = [
    {"side": d["side"], "size": float(d["contracts"]) * float(d["contractSize"]), "market_price":d["info"]["marketPrice"], "usd_size": float(d["contracts"]) * float(d["contractSize"]) * float(d["info"]["marketPrice"]), "open_price": d["entryPrice"]}
    for d in positions_data if d["symbol"] == pair]

In [10]:
positions_data

[]